In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
from  urllib.parse import quote, urlencode, urlparse
import json
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)

import requests

def make_soup(content):
    return BeautifulSoup(content, 'html.parser')

In [11]:
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 " \
             "Safari/537.36 "

keywords=["rtx", "3090"]

payload = {'lazyForm': 'lazyForm',
            'q': ' '.join(keywords),
            'lazyComponent': 'lazyListingContainer',
            'javax.faces.ViewState': 'stateless',
            'javax.faces.source': 'lazyButton',
            'javax.faces.partial.event': 'click',
            'javax.faces.partial.execute': 'lazyButton lazyButton',
            'javax.faces.behavior.event': 'action',
            'javax.faces.partial.ajax': 'true'}
headers = {
    'user-agent': USER_AGENT,
    'Origin': 'https://www.alternate.fr',
    'Referer': f'https://www.alternate.fr/listing.xhtml?q={quote(" ".join(keywords))}'
}

url = f"https://www.alternate.fr/listing.xhtml"

session = requests.Session()
session.headers.update(headers)

resp = session.get(url + f'?q={quote(" ".join(keywords))}')
resp.raise_for_status()
resp = session.post(url, data=payload, headers=headers)
resp.raise_for_status()

In [12]:
bs = make_soup(resp.content)

In [13]:
print(bs.get_text())




ALTERNATE - Le spécialiste en ligne High-Tech&!























JavaScript n'est pas activé !
              Pour pouvoir utiliser cette page, vous devez activer JavaScript dans votre navigateur.
            







































 Compte




Connexion


Créer un compte client
Mon compteCommandesConfigurations PCDonnées personnellesAdressesMa liste de favorisParamétres
Comparatif des produitsArticles consultés récemment











0
 Panier









 Toutes les catégories



PC-Builder
































                  Matériel
                

                  Gaming
                

                  Smartphone
                

                  Caméra
                

                  Tablettes
                

                  Ménage
                

                  Outils
                

                  Jardin
                

                  Extérieur
                

                  Apple
                

          

In [ ]:
resp_json = resp.json()
pp.pprint(resp_json["produits"][0])

In [72]:
metadata = bs.find("script", attrs={'type':'application/ld+json'})
pp.pprint(json.loads(metadata.string))

{   '@context': 'http://schema.org/',
    '@type': 'Product',
    'brand': {'@type': 'Brand', 'name': 'EVGA'},
    'description': 'La carte graphique EVGA GeForce RTX 3080 XC3 GAMING '
                   'embarque 10 Go de mémoire vidéo de nouvelle génération '
                   "GDDR6X. Ce modèle overclocké d'usine bénéficie de "
                   "fréquences de fonctionnement élevées et d'un système de "
                   'refroidissement amélioré gage de fiabilité et de '
                   'performances à long terme.',
    'mpn': '10G-P5-3883-KR',
    'name': 'EVGA GeForce RTX 3080 XC3 GAMING',
    'offers': {   '@type': 'Offer',
                  'availability': 'http://schema.org/OutOfStock',
                  'price': '863.95',
                  'priceCurrency': 'EUR'}}


In [90]:
import re
item = items[0]
print(item.find(class_="description").get_text())
item_id = item.attrs["id"]
script_data = ''.join(s.string for s in bs.find_all("script", attrs={"src": None}))
re.search("#{id}.*?stock-wrapper.*?stock-([0-9])".format(id=item_id), script_data)[1]


EVGA GeForce RTX 3090 FTW3 GAMING
24 Go GDDR6X - HDMI/Tri DisplayPort - PCI Express (NVIDIA GeForce RTX 3090)


Call of Duty : Black Ops Cold War offert avec NVIDIA
                    




'9'

In [77]:
target_url = "https://www.materiel.net/product-listing/stock-price/"
# target_url = "https://shop.hardware.fr/stock-price/"
headers = {
    'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    # 'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'x-requested-with': 'XMLHttpRequest'
}
# payload = "json=%7B%22currencyISOCode3%22%3A%22EUR%22%2C%22offers%22%3A%5B%7B%22offerId%22%3A%22AR202009090100%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009090101%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012070098%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009090098%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009090099%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009100088%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012070099%22%2C%22marketplace%22%3Afalse%7D%5D%2C%22shops%22%3A%5B%7B%22shopId%22%3A-1%7D%5D%7D&shopId=-1&displayGroups=Web&shopsAvailability=%7B%22AR202009090100%22%3A%220%22%2C%22AR202009090101%22%3A%220%22%2C%22AR202012070098%22%3A%220%22%2C%22AR202009090098%22%3A%220%22%2C%22AR202009090099%22%3A%220%22%2C%22AR202009100088%22%3A%220%22%2C%22AR202012070099%22%3A%220%22%7D"
# payload = "json=%7B%22currencyISOCode3%22%3A+%22EUR%22%2C+%22offers%22%3A+%5B%7B%22offerId%22%3A+%22AR202009090101%22%2C+%22marketplace%22%3A+false%7D%2C+%7B%22offerId%22%3A+%22AR202009090099%22%2C+%22marketplace%22%3A+false%7D%2C+%7B%22offerId%22%3A+%22AR202012070098%22%2C+%22marketplace%22%3A+false%7D%2C+%7B%22offerId%22%3A+%22AR202009100088%22%2C+%22marketplace%22%3A+false%7D%2C+%7B%22offerId%22%3A+%22AR202009090098%22%2C+%22marketplace%22%3A+false%7D%2C+%7B%22offerId%22%3A+%22AR202009090100%22%2C+%22marketplace%22%3A+false%7D%2C+%7B%22offerId%22%3A+%22AR202012070099%22%2C+%22marketplace%22%3A+false%7D%5D%2C+%22shops%22%3A+%5B%7B%22shopId%22%3A+-1%7D%5D%7D&shopId=-1&displayGroups=Web&shopsAvailability=%7B%22AR202009090101%22%3A+%220%22%2C+%22AR202009090099%22%3A+%220%22%2C+%22AR202012070098%22%3A+%220%22%2C+%22AR202009100088%22%3A+%220%22%2C+%22AR202009090098%22%3A+%220%22%2C+%22AR202009090100%22%3A+%220%22%2C+%22AR202012070099%22%3A+%220%22%7D"
payload = {
    "json": json.dumps({
        "currencyISOCode3":"EUR",
        "offers":[
            {"offerId":"AR202009090101","marketplace":False},
            {"offerId":"AR202009090099","marketplace":False},
            {"offerId":"AR202012070098","marketplace":False},
            {"offerId":"AR202009100088","marketplace":False},
            {"offerId":"AR202009090098","marketplace":False},
            {"offerId":"AR202009090100","marketplace":False},
            {"offerId":"AR202012070099","marketplace":False}],
        "shops":[{"shopId":-1}]}),
    # "shopId": -1,
    # "displayGroups": "Web",
    # "shopsAvailability": json.dumps({
    #     "AR202009090101":"0",
    #     "AR202009090099":"0",
    #     "AR202012070098":"0",
    #     "AR202009100088":"0",
    #     "AR202009090098":"0",
    #     "AR202009090100":"0",
    #     "AR202012070099":"0"})
}
# encoded = urlencode(payload)
# decoded = urlparse(encoded)

resp = requests.post(target_url, data=payload, headers=headers)
resp.raise_for_status()

In [78]:
pp.pprint(resp.json())

{   'nbShopsAvailability': [],
    'price': {   'AR202009090098': '<div class="o-product__prices "><span '
                                   'class="o-product__price">849€<sup>95</sup></span></div>',
                 'AR202009090099': '<div class="o-product__prices "><span '
                                   'class="o-product__price">899€<sup>95</sup></span></div>',
                 'AR202009090100': '<div class="o-product__prices "><span '
                                   'class="o-product__price">989€<sup>95</sup></span></div>',
                 'AR202009090101': '<div class="o-product__prices "><span '
                                   'class="o-product__price">999€<sup>95</sup></span></div>',
                 'AR202009100088': '<div class="o-product__prices "><span '
                                   'class="o-product__price">929€<sup>95</sup></span></div>',
                 'AR202012070098': '<div class="o-product__prices "><span '
                                   'class="

In [104]:
import re
len(json["price"])
def is_in_stock(art):
    match = re.search(r"o-availability__value--stock_([0-9])", art)
    assert match
    return int(match[1]) <= 2

any(map(is_in_stock, json["stock"].values()))

False

In [20]:
x = ['abc', None, 'cd']
y = [(1, 1), (2, 2), (3, 3)]

import itertools
def starzip(*iterables):
    def ensure_tuple(e):
        return e if isinstance(e, tuple) else (e,)
    def flatten_to_tuple(l):
        return tuple(itertools.chain.from_iterable(map(ensure_tuple, l)))

    return map(flatten_to_tuple, zip(*iterables))

list(starzip(x, y))

[('abc', 1, 1), (None, 2, 2), ('cd', 3, 3)]